# Day8_4: ARIMA/SARIMA 예측 모델링 - 정답 노트북

---

이 노트북은 Day8_4 실습 퀴즈의 정답과 상세한 풀이를 포함합니다.

## 🔧 환경 설정

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 시계열 분석 라이브러리
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
import pmdarima as pm

# 예측 성능 평가
from sklearn.metrics import mean_absolute_error, mean_squared_error

import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료!")

In [ ]:
# 공통 데이터 준비
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
df_air = pd.read_csv(url, index_col='Month', parse_dates=True)
df_air.columns = ['Passengers']

# 훈련/테스트 분할
train = df_air['Passengers'][:-12]
test = df_air['Passengers'][-12:]

print(f"훈련 데이터: {len(train)}개, 테스트 데이터: {len(test)}개")

---

## 퀴즈 1 ⭐ - ARIMA 모델 학습 및 예측

### 문제
ARIMA(2,1,0) 모델을 학습하고 12개월 예측을 수행하세요.

In [ ]:
# 퀴즈 1 정답
from statsmodels.tsa.arima.model import ARIMA

# ARIMA(2,1,0) 모델 학습
# - p=2: AR(2) - 과거 2시점 참조
# - d=1: 1차 차분 적용
# - q=0: MA 항 없음
model = ARIMA(train, order=(2, 1, 0))
fitted = model.fit()

# 12개월 예측
forecast = fitted.forecast(steps=12)
print("ARIMA(2,1,0) 예측 결과:")
print(forecast)

In [ ]:
# 테스트: 예측 결과 검증
assert len(forecast) == 12, "예측 개수가 12가 아닙니다"
assert all(forecast > 0), "예측값은 양수여야 합니다"
print("테스트 통과!")

### 💡 풀이 설명

**핵심 개념:**
- `order=(p, d, q)` 튜플로 ARIMA 차수 지정
- `model.fit()`으로 모델 학습
- `fitted.forecast(steps=n)`으로 n개 예측

**ARIMA(2,1,0) 해석:**
- AR(2): 현재값 = 이전 2시점의 가중합 + 오차
- I(1): 원본 대신 1차 차분 데이터로 모델링
- MA(0): 이동평균 항 없음 (순수 AR 모델)

---

## 퀴즈 2 ⭐ - 스케일 독립적 지표 (MAPE)

### 문제
MAE, RMSE, MAPE 중 스케일에 독립적인 지표를 계산하세요.

In [ ]:
# 퀴즈 2 정답
import numpy as np

actual = np.array([100, 150, 200, 180])
predicted = np.array([110, 140, 190, 170])

# MAPE (Mean Absolute Percentage Error) 계산
# 스케일 독립적 → 백분율로 표현되어 다른 데이터셋과 비교 가능
mape = np.mean(np.abs((actual - predicted) / actual)) * 100
print(f"MAPE: {mape:.2f}%")

In [ ]:
# 테스트: MAPE 검증
# 수동 계산: |100-110|/100 + |150-140|/150 + |200-190|/200 + |180-170|/180
#         = 0.1 + 0.0667 + 0.05 + 0.0556 = 0.2722 → 27.22%
expected_mape = (10/100 + 10/150 + 10/200 + 10/180) / 4 * 100
assert abs(mape - expected_mape) < 0.01, "MAPE 계산 오류"
print(f"검증값: {expected_mape:.2f}% - 테스트 통과!")

### 💡 풀이 설명

**스케일 독립적 지표가 중요한 이유:**
- MAE: 원본 스케일 (주가 100원 vs 100만원 비교 불가)
- RMSE: 원본 스케일 (동일 문제)
- **MAPE**: 백분율 → 스케일 무관하게 비교 가능

**MAPE 공식:**
$$MAPE = \frac{100}{n} \sum_{i=1}^{n} \left| \frac{y_i - \hat{y}_i}{y_i} \right|$$

**주의:** 실제값이 0이면 분모가 0이 되어 계산 불가 (대안: SMAPE)

---

## 퀴즈 3 ⭐⭐ - ACF/PACF 플롯 해석

### 문제
PACF lag 2 이후 절단, ACF 점차 감소 패턴에 적합한 ARIMA 차수를 결정하세요.

In [ ]:
# 퀴즈 3 정답

# ACF/PACF 해석 규칙:
# - PACF lag k 이후 절단 → AR(k) 모델
# - ACF lag k 이후 절단 → MA(k) 모델
# - 둘 다 점차 감소 → ARMA 혼합 모델

# PACF 절단: lag 2 이후 → p = 2
# ACF 점차 감소 → 순수 AR 모델 (q = 0)

p = 2  # PACF가 lag 2에서 절단
d = 1  # 정상성을 위한 차분 (보통 1)
q = 0  # ACF가 점차 감소하면 MA 항 불필요

print(f"적합한 모델: ARIMA({p},{d},{q})")
print("해석: AR(2) 모델 - 과거 2시점의 값이 현재에 직접 영향")

In [ ]:
# 테스트: 차수 검증
assert p == 2, "p 값 오류: PACF lag 2 절단이면 p=2"
assert q == 0, "q 값 오류: ACF 점차 감소면 순수 AR 모델 (q=0)"
print("테스트 통과!")

### 💡 풀이 설명

**ACF/PACF 해석 정리표:**

| 패턴 | PACF | ACF | 모델 |
|------|------|-----|------|
| 순수 AR(p) | lag p 이후 절단 | 점차 감소 | ARIMA(p,d,0) |
| 순수 MA(q) | 점차 감소 | lag q 이후 절단 | ARIMA(0,d,q) |
| ARMA(p,q) | 점차 감소 | 점차 감소 | ARIMA(p,d,q) |

**절단의 의미:**
- 특정 lag 이후 갑자기 신뢰구간(파란 영역) 안으로 떨어지는 것
- 점차 감소: 지수적으로 천천히 0에 수렴

---

## 퀴즈 4 ⭐⭐ - ARIMA(1,1,1) 전체 파이프라인

### 문제
항공 승객 데이터로 ARIMA(1,1,1) 학습, 12개월 예측, RMSE 계산

In [ ]:
# 퀴즈 4 정답
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np

# 데이터 로드
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
df = pd.read_csv(url, index_col='Month', parse_dates=True)
df.columns = ['Passengers']
series = df['Passengers']

# 훈련/테스트 분할
train = series[:-12]
test = series[-12:]

# 모델 학습
model = ARIMA(train, order=(1, 1, 1))
fitted = model.fit()

# 예측
forecast = fitted.forecast(steps=12)

# RMSE 계산
rmse = np.sqrt(mean_squared_error(test.values, forecast.values))

print(f"RMSE: {rmse:.2f}")
print(f"\n예측 vs 실제 비교:")
comparison = pd.DataFrame({
    '실제값': test.values,
    '예측값': forecast.values,
    '오차': test.values - forecast.values
}, index=test.index)
print(comparison)

In [ ]:
# 테스트: RMSE 범위 확인
assert rmse > 0, "RMSE는 양수여야 합니다"
assert rmse < 100, "RMSE가 비정상적으로 큽니다"
print("테스트 통과!")

### 💡 풀이 설명

**전체 파이프라인 순서:**
1. 데이터 로드 및 전처리
2. 훈련/테스트 분할 (시계열은 시간순 분할!)
3. ARIMA 모델 학습 (`ARIMA().fit()`)
4. 예측 수행 (`forecast(steps=n)`)
5. 성능 평가 (RMSE 등)

**시계열 분할 주의:**
- 랜덤 분할 X → 시간 순서대로 분할
- 미래 데이터로 과거를 예측하는 데이터 누수 방지

---

## 퀴즈 5 ⭐⭐ - 계절 주기(m) 설정

### 문제
월별, 분기별, 주간 데이터의 계절 주기 m 값을 결정하세요.

In [ ]:
# 퀴즈 5 정답

# 계절 주기(m): 한 주기를 완성하는 데이터 포인트 수

# 월별 데이터: 1년 = 12개월 → m = 12
# 예: 매년 여름 판매량 패턴 반복
m_monthly = 12

# 분기별 데이터: 1년 = 4분기 → m = 4
# 예: 분기별 실적 발표 패턴
m_quarterly = 4

# 주간 데이터: 1주 = 7일 → m = 7
# 예: 매주 같은 요일 패턴 (월요일 = 저조, 주말 = 활성)
m_weekly = 7

print(f"월별: m={m_monthly}")
print(f"분기별: m={m_quarterly}")
print(f"주간: m={m_weekly}")

# 추가 예시
print("\n추가 예시:")
print(f"시간별 (일간 패턴): m=24")
print(f"15분 단위 (일간 패턴): m=96 (24*4)")

In [ ]:
# 테스트
assert m_monthly == 12, "월별 주기는 12"
assert m_quarterly == 4, "분기별 주기는 4"
assert m_weekly == 7, "주간 주기는 7"
print("테스트 통과!")

### 💡 풀이 설명

**계절 주기 결정 방법:**

| 데이터 주기 | 계절 패턴 | m 값 |
|------------|----------|------|
| 월별 | 연간 | 12 |
| 분기별 | 연간 | 4 |
| 주간 | 주간 | 7 |
| 일별 | 주간 | 7 |
| 시간별 | 일간 | 24 |

**실무 팁:**
- ACF 플롯에서 주기적 스파이크 위치로도 확인 가능
- lag 12에서 스파이크 → 12개월 계절성
- 여러 계절성이 있을 수 있음 (일간 + 주간 + 연간)

---

## 퀴즈 6 ⭐⭐⭐ - auto_arima SARIMA

### 문제
auto_arima로 최적 SARIMA 모델을 찾고 성능을 평가하세요.

In [ ]:
# 퀴즈 6 정답
import pmdarima as pm

# auto_arima로 최적 SARIMA 모델 탐색
model = pm.auto_arima(
    train,
    seasonal=True,    # 계절성 모델 사용
    m=12,             # 계절 주기 (월별 데이터)
    D=1,              # 계절 차분 1회 고정
    start_p=0, start_q=0,
    max_p=3, max_q=3,
    trace=True,       # 탐색 과정 출력
    error_action='ignore',
    suppress_warnings=True,
    stepwise=True
)

print(f"\n최적 모델: SARIMA{model.order}{model.seasonal_order}")
print(f"AIC: {model.aic():.2f}")

In [ ]:
# 예측 및 성능 평가
forecast_sarima, conf_int = model.predict(n_periods=12, return_conf_int=True)

# MAE, RMSE, MAPE 계산
mae = mean_absolute_error(test.values, forecast_sarima)
rmse = np.sqrt(mean_squared_error(test.values, forecast_sarima))
mape = np.mean(np.abs((test.values - forecast_sarima) / test.values)) * 100

print(f"\n=== 예측 성능 ===")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
# 테스트
assert model.seasonal_order[3] == 12, "계절 주기가 12가 아닙니다"
assert mae > 0 and rmse > 0, "MAE, RMSE는 양수여야 합니다"
print("테스트 통과!")

### 💡 풀이 설명

**auto_arima 주요 파라미터:**
- `seasonal=True`: 계절성 모델 활성화
- `m=12`: 계절 주기 (연간 = 12개월)
- `D=1`: 계절 차분 횟수 (보통 1로 고정)
- `trace=True`: 탐색 과정 출력
- `stepwise=True`: 효율적 탐색 알고리즘

**AIC (Akaike Information Criterion):**
- 모델 복잡도와 적합도의 균형
- 낮을수록 좋은 모델
- 과적합 방지 역할

---

## 퀴즈 7 ⭐⭐⭐ - 주가 예측 파이프라인

### 문제
애플(AAPL) 주가를 수집하고 ARIMA 예측 파이프라인을 구축하세요.

In [ ]:
# 퀴즈 7 정답
import yfinance as yf
import pmdarima as pm

# 1. 데이터 수집
ticker = "AAPL"
df_stock = yf.download(ticker, period="1y")
series = df_stock['Close'].dropna()

print(f"데이터 기간: {series.index[0].date()} ~ {series.index[-1].date()}")
print(f"데이터 개수: {len(series)}")

In [ ]:
# 2. 훈련/테스트 분할
test_days = 30
train_stock = series[:-test_days]
test_stock = series[-test_days:]

print(f"훈련: {len(train_stock)}일, 테스트: {len(test_stock)}일")

In [ ]:
# 3. auto_arima 모델 (비계절성 - 일별 주가)
model_stock = pm.auto_arima(
    train_stock,
    start_p=0, start_q=0,
    max_p=5, max_q=5,
    d=None,  # 자동 결정
    seasonal=False,  # 일별 데이터는 비계절성
    trace=True,
    suppress_warnings=True,
    stepwise=True
)

print(f"\n최적 모델: ARIMA{model_stock.order}")

In [ ]:
# 4. 예측 및 시각화
forecast_stock, conf_int_stock = model_stock.predict(
    n_periods=test_days,
    return_conf_int=True
)

# Plotly 시각화
fig = go.Figure()

# 훈련 데이터 (최근 60일만)
recent_train = train_stock[-60:]
fig.add_trace(go.Scatter(
    x=recent_train.index, y=recent_train.values,
    mode='lines', name='훈련 데이터',
    line=dict(color='blue')
))

# 실제 테스트 데이터
fig.add_trace(go.Scatter(
    x=test_stock.index, y=test_stock.values,
    mode='lines', name='실제값',
    line=dict(color='green')
))

# 예측값
fig.add_trace(go.Scatter(
    x=test_stock.index, y=forecast_stock,
    mode='lines', name='예측값',
    line=dict(color='red', dash='dash')
))

# 신뢰구간
fig.add_trace(go.Scatter(
    x=list(test_stock.index) + list(test_stock.index[::-1]),
    y=list(conf_int_stock[:, 1]) + list(conf_int_stock[::-1, 0]),
    fill='toself',
    fillcolor='rgba(255,0,0,0.2)',
    line=dict(color='rgba(255,0,0,0)'),
    name='95% 신뢰구간'
))

fig.update_layout(
    title=f'{ticker} 주가 예측 (ARIMA{model_stock.order})',
    xaxis_title='날짜',
    yaxis_title='종가 (USD)',
    legend=dict(x=0.02, y=0.98)
)
fig.show()

In [ ]:
# 성능 평가
mae_stock = mean_absolute_error(test_stock.values, forecast_stock)
rmse_stock = np.sqrt(mean_squared_error(test_stock.values, forecast_stock))
mape_stock = np.mean(np.abs((test_stock.values - forecast_stock) / test_stock.values)) * 100

print(f"=== {ticker} 예측 성능 ===")
print(f"MAE: ${mae_stock:.2f}")
print(f"RMSE: ${rmse_stock:.2f}")
print(f"MAPE: {mape_stock:.2f}%")

### 💡 풀이 설명

**주가 예측 파이프라인:**
1. `yf.download()`: 야후 파이낸스에서 주가 수집
2. 종가(Close) 시리즈 추출
3. auto_arima (비계절성): 일별 데이터는 명확한 계절성 없음
4. 신뢰구간 포함 시각화

**실무 주의:**
- 주가는 이론적으로 랜덤워크 (예측 어려움)
- ARIMA는 단기 추세 파악용
- 투자 결정에 단독 사용 금물

---

## 퀴즈 8 ⭐⭐⭐⭐ - ARIMA 차수 비교 함수

### 문제
여러 ARIMA 차수를 비교하는 함수를 구현하세요.

In [ ]:
# 퀴즈 8 정답
def compare_arima_models(train, test, orders):
    """
    여러 ARIMA 차수를 비교하고 결과를 DataFrame으로 반환
    
    Parameters:
    -----------
    train : pd.Series
        훈련 데이터
    test : pd.Series
        테스트 데이터
    orders : list of tuples
        비교할 (p,d,q) 조합 리스트
    
    Returns:
    --------
    pd.DataFrame : 모델별 성능 비교 결과
    """
    results = []
    
    for order in orders:
        try:
            # 모델 학습
            model = ARIMA(train, order=order)
            fitted = model.fit()
            
            # 예측
            forecast = fitted.forecast(steps=len(test))
            
            # 성능 지표 계산
            mae = mean_absolute_error(test.values, forecast.values)
            rmse = np.sqrt(mean_squared_error(test.values, forecast.values))
            mape = np.mean(np.abs((test.values - forecast.values) / test.values)) * 100
            
            results.append({
                '모델': f'ARIMA{order}',
                'AIC': round(fitted.aic, 2),
                'BIC': round(fitted.bic, 2),
                'MAE': round(mae, 2),
                'RMSE': round(rmse, 2),
                'MAPE': round(mape, 2)
            })
        except Exception as e:
            # 학습 실패 시 스킵
            print(f"ARIMA{order} 학습 실패: {e}")
            continue
    
    # DataFrame으로 변환 후 AIC 기준 정렬
    df_results = pd.DataFrame(results)
    return df_results.sort_values('AIC').reset_index(drop=True)

# 테스트
orders = [(1,1,0), (0,1,1), (1,1,1), (2,1,1), (1,1,2), (2,1,2)]
comparison = compare_arima_models(train, test, orders)
print("=== ARIMA 차수 비교 결과 ===")
comparison

In [ ]:
# 테스트
assert len(comparison) > 0, "결과가 없습니다"
assert 'AIC' in comparison.columns, "AIC 열이 없습니다"
assert comparison['AIC'].is_monotonic_increasing or len(comparison) == 1, "AIC 정렬 오류"
print("테스트 통과!")

### 💡 풀이 설명

**함수 설계 핵심:**
1. 예외 처리: 일부 차수는 학습 실패 가능 (수렴 안 됨)
2. 여러 지표 계산: AIC, BIC, MAE, RMSE, MAPE
3. 결과 정렬: AIC 기준 오름차순

**모델 선택 기준:**
- AIC/BIC 낮은 순 (모델 복잡도 고려)
- 실제 예측 성능 (MAE, RMSE, MAPE) 종합 판단
- 해석 용이성 (간단한 모델 선호)

---

## 퀴즈 9 ⭐⭐⭐⭐ - 롤링 예측 구현

### 문제
한 시점씩 예측하고 실제값을 추가하는 롤링 예측을 구현하세요.

In [ ]:
# 퀴즈 9 정답
def rolling_forecast(series, test_size, order):
    """
    롤링(Walk-Forward) 예측 수행
    
    각 시점에서:
    1. 현재까지의 데이터로 모델 학습
    2. 1시점 예측
    3. 실제값을 훈련 데이터에 추가
    4. 반복
    
    Parameters:
    -----------
    series : pd.Series
        전체 시계열 데이터
    test_size : int
        테스트 기간 (롤링 예측 횟수)
    order : tuple
        ARIMA 차수 (p, d, q)
    
    Returns:
    --------
    list : 롤링 예측값 리스트
    """
    # 훈련 데이터를 리스트로 변환 (점진적 추가를 위해)
    history = list(series[:-test_size])
    predictions = []
    
    for i in range(test_size):
        # 1. 현재 history로 모델 학습
        model = ARIMA(history, order=order)
        fitted = model.fit()
        
        # 2. 1시점 예측
        yhat = fitted.forecast(steps=1)[0]
        predictions.append(yhat)
        
        # 3. 실제값을 history에 추가
        actual = series.iloc[-(test_size - i)]
        history.append(actual)
        
        # 진행 상황 출력 (선택사항)
        if (i + 1) % 4 == 0:
            print(f"진행: {i+1}/{test_size}")
    
    return predictions

# 롤링 예측 실행
print("롤링 예측 수행 중...")
rolling_preds = rolling_forecast(df_air['Passengers'], test_size=12, order=(1,1,1))
print("완료!")

In [ ]:
# 롤링 예측 vs 일괄 예측 비교
# 일괄 예측
batch_model = ARIMA(train, order=(1,1,1)).fit()
batch_preds = batch_model.forecast(steps=12).values

# 성능 비교
print("=== 예측 방식 비교 ===")

# 롤링 예측 성능
rolling_mae = mean_absolute_error(test.values, rolling_preds)
rolling_rmse = np.sqrt(mean_squared_error(test.values, rolling_preds))
print(f"\n롤링 예측:")
print(f"  MAE: {rolling_mae:.2f}")
print(f"  RMSE: {rolling_rmse:.2f}")

# 일괄 예측 성능
batch_mae = mean_absolute_error(test.values, batch_preds)
batch_rmse = np.sqrt(mean_squared_error(test.values, batch_preds))
print(f"\n일괄 예측:")
print(f"  MAE: {batch_mae:.2f}")
print(f"  RMSE: {batch_rmse:.2f}")

# 개선율
improvement = (batch_mae - rolling_mae) / batch_mae * 100
print(f"\n롤링 예측 MAE 개선율: {improvement:.1f}%")

In [ ]:
# 시각화 비교
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=test.index, y=test.values,
    mode='lines+markers', name='실제값',
    line=dict(color='black', width=2)
))

fig.add_trace(go.Scatter(
    x=test.index, y=rolling_preds,
    mode='lines+markers', name='롤링 예측',
    line=dict(color='blue', dash='dash')
))

fig.add_trace(go.Scatter(
    x=test.index, y=batch_preds,
    mode='lines+markers', name='일괄 예측',
    line=dict(color='red', dash='dot')
))

fig.update_layout(
    title='롤링 예측 vs 일괄 예측 비교',
    xaxis_title='날짜',
    yaxis_title='승객 수'
)
fig.show()

### 💡 풀이 설명

**롤링 예측 vs 일괄 예측:**

| 방식 | 특징 | 장점 | 단점 |
|------|------|------|------|
| **일괄 예측** | 한 번 학습 후 n개 예측 | 빠름 | 멀수록 부정확 |
| **롤링 예측** | 매 시점 재학습 | 최신 정보 반영 | 느림 |

**롤링 예측이 더 정확한 이유:**
- 최신 실제값을 학습에 반영
- 단기 예측 (1시점)만 수행
- 실시간 예측 시스템에 적합

**실무 활용:**
- 온라인 학습 시스템
- 실시간 예측 서비스
- 주식/날씨 예측

---

## 퀴즈 10 ⭐⭐⭐⭐⭐ - TimeSeriesForecaster 클래스

### 문제
시계열 예측 파이프라인을 클래스로 구현하세요.

In [ ]:
# 퀴즈 10 정답
class TimeSeriesForecaster:
    """
    시계열 예측 파이프라인 클래스
    
    auto_arima를 사용하여 최적 모델을 자동으로 선택하고,
    예측, 평가, 시각화 기능을 제공합니다.
    
    Parameters:
    -----------
    seasonal : bool
        계절성 모델 사용 여부 (default: False)
    m : int
        계절 주기 (default: 12)
    
    Attributes:
    -----------
    model : pmdarima.ARIMA
        학습된 ARIMA/SARIMA 모델
    train : pd.Series
        훈련 데이터
    test : pd.Series
        테스트 데이터
    forecast : array
        예측 결과
    conf_int : array
        신뢰구간
    
    Methods:
    --------
    fit(series, test_size) : 모델 학습
    predict(steps) : 예측 수행
    evaluate() : 성능 평가
    plot() : 결과 시각화
    """
    
    def __init__(self, seasonal=False, m=12):
        self.seasonal = seasonal
        self.m = m
        self.model = None
        self.train = None
        self.test = None
        self.forecast = None
        self.conf_int = None
        self._is_fitted = False
    
    def fit(self, series, test_size=12):
        """
        데이터 분할 및 최적 모델 학습
        
        Parameters:
        -----------
        series : pd.Series
            전체 시계열 데이터
        test_size : int
            테스트 데이터 크기
        
        Returns:
        --------
        self : 학습된 모델 객체
        """
        # 데이터 분할
        self.train = series[:-test_size]
        self.test = series[-test_size:]
        
        print(f"데이터 분할: 훈련 {len(self.train)}개, 테스트 {len(self.test)}개")
        print(f"모델 유형: {'SARIMA' if self.seasonal else 'ARIMA'}")
        print("\n최적 모델 탐색 중...")
        
        # auto_arima로 최적 모델 탐색
        self.model = pm.auto_arima(
            self.train,
            start_p=0, start_q=0,
            max_p=5, max_q=5,
            seasonal=self.seasonal,
            m=self.m if self.seasonal else 1,
            D=1 if self.seasonal else None,
            trace=False,
            error_action='ignore',
            suppress_warnings=True,
            stepwise=True
        )
        
        self._is_fitted = True
        
        if self.seasonal:
            print(f"최적 모델: SARIMA{self.model.order}{self.model.seasonal_order}")
        else:
            print(f"최적 모델: ARIMA{self.model.order}")
        print(f"AIC: {self.model.aic():.2f}")
        
        return self
    
    def predict(self, steps=None):
        """
        예측 수행
        
        Parameters:
        -----------
        steps : int, optional
            예측 시점 수 (default: test_size)
        
        Returns:
        --------
        forecast : array
            예측값
        """
        if not self._is_fitted:
            raise ValueError("먼저 fit() 메서드를 호출하세요.")
        
        if steps is None:
            steps = len(self.test)
        
        self.forecast, self.conf_int = self.model.predict(
            n_periods=steps,
            return_conf_int=True
        )
        
        print(f"{steps}시점 예측 완료")
        return self.forecast
    
    def evaluate(self):
        """
        예측 성능 평가
        
        Returns:
        --------
        metrics : dict
            MAE, RMSE, MAPE 성능 지표
        """
        if self.forecast is None:
            raise ValueError("먼저 predict() 메서드를 호출하세요.")
        
        actual = self.test.values[:len(self.forecast)]
        
        mae = mean_absolute_error(actual, self.forecast)
        rmse = np.sqrt(mean_squared_error(actual, self.forecast))
        mape = np.mean(np.abs((actual - self.forecast) / actual)) * 100
        
        metrics = {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}
        
        print("\n=== 예측 성능 ===")
        print(f"MAE: {mae:.2f}")
        print(f"RMSE: {rmse:.2f}")
        print(f"MAPE: {mape:.2f}%")
        
        return metrics
    
    def plot(self, show_train_days=60):
        """
        예측 결과 시각화
        
        Parameters:
        -----------
        show_train_days : int
            표시할 훈련 데이터 일수 (default: 60)
        
        Returns:
        --------
        fig : plotly.graph_objects.Figure
        """
        if self.forecast is None:
            raise ValueError("먼저 predict() 메서드를 호출하세요.")
        
        fig = go.Figure()
        
        # 훈련 데이터 (최근 n일)
        recent_train = self.train[-show_train_days:]
        fig.add_trace(go.Scatter(
            x=recent_train.index, y=recent_train.values,
            mode='lines', name='훈련 데이터',
            line=dict(color='blue')
        ))
        
        # 테스트 데이터 (실제값)
        fig.add_trace(go.Scatter(
            x=self.test.index, y=self.test.values,
            mode='lines', name='실제값',
            line=dict(color='green')
        ))
        
        # 예측값
        fig.add_trace(go.Scatter(
            x=self.test.index[:len(self.forecast)], y=self.forecast,
            mode='lines', name='예측값',
            line=dict(color='red', dash='dash')
        ))
        
        # 신뢰구간
        if self.conf_int is not None:
            forecast_idx = self.test.index[:len(self.forecast)]
            fig.add_trace(go.Scatter(
                x=list(forecast_idx) + list(forecast_idx[::-1]),
                y=list(self.conf_int[:, 1]) + list(self.conf_int[::-1, 0]),
                fill='toself',
                fillcolor='rgba(255,0,0,0.2)',
                line=dict(color='rgba(255,0,0,0)'),
                name='95% 신뢰구간'
            ))
        
        # 레이아웃
        model_name = f"SARIMA{self.model.order}{self.model.seasonal_order}" if self.seasonal else f"ARIMA{self.model.order}"
        fig.update_layout(
            title=f'시계열 예측 결과 ({model_name})',
            xaxis_title='날짜',
            yaxis_title='값',
            legend=dict(x=0.02, y=0.98)
        )
        
        fig.show()
        return fig
    
    def summary(self):
        """모델 요약 정보 출력"""
        if not self._is_fitted:
            print("모델이 학습되지 않았습니다.")
            return
        print(self.model.summary())

In [ ]:
# 사용 예시: SARIMA (계절성 O)
print("=" * 50)
print("TimeSeriesForecaster 클래스 사용 예시")
print("=" * 50)

# 인스턴스 생성
forecaster = TimeSeriesForecaster(seasonal=True, m=12)

# 모델 학습
forecaster.fit(df_air['Passengers'], test_size=12)

# 예측
forecaster.predict()

# 성능 평가
metrics = forecaster.evaluate()

# 시각화
forecaster.plot(show_train_days=48)

In [ ]:
# 사용 예시: ARIMA (계절성 X)
print("\n" + "=" * 50)
print("비계절성 ARIMA 예시")
print("=" * 50)

forecaster_arima = TimeSeriesForecaster(seasonal=False)
forecaster_arima.fit(df_air['Passengers'], test_size=12)
forecaster_arima.predict()
metrics_arima = forecaster_arima.evaluate()

In [ ]:
# 테스트: 클래스 기능 검증
assert forecaster._is_fitted, "모델이 학습되지 않았습니다"
assert forecaster.forecast is not None, "예측이 수행되지 않았습니다"
assert 'MAE' in metrics, "MAE가 반환되지 않았습니다"
assert 'RMSE' in metrics, "RMSE가 반환되지 않았습니다"
assert 'MAPE' in metrics, "MAPE가 반환되지 않았습니다"
print("테스트 통과!")

### 💡 풀이 설명

**클래스 설계 원칙:**

1. **캡슐화**: 관련 기능을 하나의 클래스로 묶음
2. **명확한 인터페이스**: fit → predict → evaluate → plot 순서
3. **상태 관리**: `_is_fitted` 플래그로 학습 상태 추적
4. **에러 처리**: 순서 위반 시 명확한 에러 메시지

**메서드별 역할:**

| 메서드 | 역할 | 반환값 |
|--------|------|--------|
| `fit()` | 데이터 분할 + 모델 학습 | self |
| `predict()` | n시점 예측 | forecast array |
| `evaluate()` | MAE, RMSE, MAPE 계산 | metrics dict |
| `plot()` | Plotly 시각화 | Figure |
| `summary()` | 모델 상세 정보 | None (출력) |

**확장 아이디어:**
- `save()` / `load()`: 모델 저장/불러오기
- `cross_validate()`: 시계열 교차 검증
- `rolling_forecast()`: 롤링 예측 지원

---

## 📊 정답 노트북 요약

| 퀴즈 | 난이도 | 핵심 개념 |
|------|--------|----------|
| 1 | ⭐ | ARIMA 모델 학습 및 예측 |
| 2 | ⭐ | MAPE (스케일 독립적 지표) |
| 3 | ⭐⭐ | ACF/PACF 해석으로 p, q 결정 |
| 4 | ⭐⭐ | ARIMA 전체 파이프라인 |
| 5 | ⭐⭐ | SARIMA 계절 주기 (m) 설정 |
| 6 | ⭐⭐⭐ | auto_arima SARIMA |
| 7 | ⭐⭐⭐ | 주가 예측 파이프라인 |
| 8 | ⭐⭐⭐⭐ | ARIMA 차수 비교 함수 |
| 9 | ⭐⭐⭐⭐ | 롤링 예측 구현 |
| 10 | ⭐⭐⭐⭐⭐ | TimeSeriesForecaster 클래스 |